In [1]:
pip install scikit-surprise

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-win_amd64.whl size=1291191 sha256=dc025f026b0e226a984995fe84ac794887f976304eedb96979076e118794cf0e
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\75\fa\bc\739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNWithMeans
# Step 1: Prepare the dataset
ratings_dict = { 
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1], 
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'], 
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3], 
}
# Convert the dictionary to a DataFrame
df = pd.DataFrame(ratings_dict)
# Use the Reader class to define the rating scale
reader = Reader(rating_scale=(1, 5))
# Load the data into the Surprise library format
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
# Step 2: Implement Item-Based Collaborative Filtering
# Define the similarity options for KNN
sim_options = {
    "name": "cosine",  # Use cosine similarity
    "user_based": False,  # Item-based filtering (user_based=False)
}
# Initialize the KNNWithMeans algorithm
algo = KNNWithMeans(sim_options=sim_options)
# Build the training set and fit the model
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
# Step 3: Predict Ratings and Recommend Movies
# Function to recommend top N movies for a user
def recommend_movies(user_id, n_recommendations=3):
    # Get a list of all items (movies)
    all_items = set(df['item'])
    # Get the movies the user has already rated
    rated_items = set(df[df['user'] == user_id]['item'])
    # Print out the rated items for the user (debugging)
    print(f"Rated items for user '{user_id}': {rated_items}")
    # Find the unrated items
    unrated_items = all_items - rated_items
    print(f"Unrated items for user '{user_id}': {unrated_items}")
    # If there are no unrated items, return a message
    if not unrated_items:
        print(f"User '{user_id}' has rated all available movies.")
        return []
    # Predict ratings for all movies that the user hasn't rated
    predictions = []
    for item in unrated_items:
        prediction = algo.predict(user_id, item)
        print(f"Predicted rating for movie {item}: {prediction.est}")  # Debugging
        predictions.append((item, prediction.est))
    # Sort the predictions by the estimated rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)
    # Return the top N recommendations
    return predictions[:n_recommendations]
# Example: Recommend top 3 movies for user 'A'
recommendations = recommend_movies('A', n_recommendations=3)
if recommendations:
    print("Top 3 recommended movies for user 'A':")
    for movie, rating in recommendations:
        print(f"Movie {movie} with predicted rating {rating:.2f}")
else:
    print("No recommendations available.")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Rated items for user 'A': {1, 2}
Unrated items for user 'A': set()
User 'A' has rated all available movies.
No recommendations available.
